 # 1. DEFINE I/O DATA FRAMES

In [17]:
import numpy as np 
import pandas as pd 

MODEL_NAME='bilstm_crf'
HPARAMS_STR='lr5.00e-04_hs512x64_ctx-p1'
MODEL_V= '17'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'evaluation_valid.csv')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/bilstm_crf/lr5.00e-04_hs512x64_ctx-p1/17/Yhat_valid.csv


In [18]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 17) (12298, 2)


In [19]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 19)


In [20]:
df.to_csv(TARGET_PATH)

In [21]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,FUNC,M_R,CTX_P-3,CTX_P-2,CTX_P-1,CTX_P+1,CTX_P+2,CTX_P+3,ARG_0,ARG_1,Y_0,Y_1
IDX,,,,,,,,,,,,,,,,,,,
123846,1,2879,5100,0,Entre,entre,estar,-,0,entre,o,convidado,o,-,rei,(A2*,A2,(A2*,A2
123847,2,2879,5100,0,os,o,estar,-,0,entre,o,convidado,o,-,rei,*,A2,*,A2
123848,3,2879,5100,0,convidados,convidado,estar,-,0,entre,o,convidado,o,-,rei,*),A2,*,A2
123849,4,2879,5100,0,estão,estar,estar,estar,1,entre,o,convidado,o,-,rei,(V*),V,*,A2
123850,5,2879,5100,0,o,o,estar,-,1,entre,o,convidado,o,-,rei,(A1*,A1,*,A2


In [22]:
df2= df[['ARG_1','Y_1']]
table= pd.pivot_table(df2.iloc[:30,:], index='ARG_1', columns='Y_1', aggfunc=len, fill_value=0)
table

Y_1,*,A0,A2
ARG_1,,,
*,1,0,0
A0,0,1,0
A1,11,0,13
A2,0,0,3
V,0,0,1


 # 2. CONFUSION MATRIX

In [23]:
df1= df[['ARG_1','Y_1']]
table= pd.pivot_table(df1, index='ARG_1', columns='Y_1', aggfunc=len, fill_value=0)

#Makes a unique index out of ground truth and predictions, filling with zeros
new_index= table.index.union(table.columns)
table= table.reindex(new_index, fill_value=0)
table.columns.names=['predictions']
table.index.name='ground truth'
table


predictions,*,A0,A1,A2,A3,A4,AM-CAU,AM-DIS,AM-EXT,AM-LOC,AM-MNR,AM-NEG,AM-PNC,AM-PRD,AM-TMP,C-AM-CAU,C-V,V
ground truth,,,,,,,,,,,,,,,,,,
*,4148,176,444,214,0,3,26,2,0,45,15,14,0,0,47,0,0,2
A0,382,441,18,27,0,0,5,1,0,6,0,1,1,0,22,1,2,4
A1,1480,89,895,312,0,0,8,0,2,50,12,0,16,2,33,0,4,6
A2,304,10,191,550,0,0,7,0,0,17,7,0,0,0,18,0,0,1
A3,25,0,21,12,13,0,0,0,0,3,0,0,0,0,8,0,0,1
A4,14,0,5,14,0,5,0,0,0,3,0,0,0,0,4,0,0,0
AM-ADV,77,2,9,11,0,0,0,0,0,0,6,0,0,0,4,0,0,0
AM-CAU,66,8,19,24,0,0,35,0,0,0,0,0,0,0,1,0,0,0
AM-DIR,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


 ## 3 Evaluation
 ### Precision .: How many selected items are relevant?
 ### Recall .: How many relevant items are selected?

In [8]:
# BASE MATRICES
diag = np.diag(table.as_matrix()).astype(np.float32)
retrieved= np.sum(table.as_matrix(), axis=0, keepdims=False).astype(np.float32) #preserve columns, sum over rows
relevant= np.sum(table.as_matrix(), axis=1, keepdims=False).astype(np.float32) #preserve rows, sum over columns

#PRECISION
P=np.nan_to_num(diag / retrieved)
precisiondf= pd.DataFrame(data= P, index=table.index, columns=['precision']).fillna(0)
precisiondf.index.name= 'ARG'
# precisiondf

#RECALL
R= np.nan_to_num(diag / relevant)
recalldf= pd.DataFrame(data= R, index=table.index, columns=['recall']).fillna(0)
recalldf.index.name= 'ARG'
# recalldf

#F1
F1= np.nan_to_num(2* ((P * R)/(P + R)))
f1scoredf= pd.DataFrame(data= F1, index=table.index, columns=['f1 score']).fillna(0)
f1scoredf.index.name= 'ARG'

#Concatenate
evaluationdf= pd.concat((precisiondf, recalldf, f1scoredf), axis=1)
evaluationdf








ValueError: Shape of passed values is (1, 2), indices imply (1, 25)

## 3.1 Total scores 

In [67]:
df2= df1[['ARG_Y']]
df2['N_ARGS']=df2.groupby('ARG_Y')['ARG_Y'].transform('size')
df2.drop_duplicates(subset='ARG_Y', inplace=True)
# df2
df2.set_index('ARG_Y', inplace=True)
df2=df2.reindex(index=new_index, fill_value=0)
df2
# Get the weights for each
weights= df2['N_ARGS'].as_matrix()

#Totals
#precision
p=100*P.dot(weights)/np.sum(weights)
print('precision: {:.2f}\%'.format(p))
#recall
r=100*R.dot(weights)/np.sum(weights)
print('recall: {:.2f}\%'.format(r))
#F1 score
print('f1 score: {:.2f}\%'.format(2*(p*r)/(p+r)))



precision: 56.79\%
recall: 58.25\%
f1 score: 57.51\%
